# Pedri Detection with YOLOv5

This notebook detects FC Barcelona's Pedri in match footage using YOLOv5. It includes data augmentation, YOLOv5 setup, model training, and video detection—all in one place for an educational exploration of sports analytics.

## Overview
- **Purpose**: Detect and track Pedri in a Barcelona game video.
- **Tools**: YOLOv5, Albumentations, OpenCV.
- **Status**: Functional but needs improvement (sporadic Pedri detection).

## Requirements
- Run in a GPU-enabled environment (e.g., Kaggle with T4).
- Video and full dataset not included—adapt paths as needed.

## 1. Data Augmentation

Augments the training dataset to increase variety.

In [1]:
import albumentations as A
import cv2
import os

# Define transformation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Blur(blur_limit=3, p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Input directories (read-only)
input_img_dir = "/kaggle/input/pedri-dataset/images/train"
input_label_dir = "/kaggle/input/pedri-dataset/labels/train"

# Output directories (writable)
output_img_dir = "/kaggle/working/pedri-dataset/images/train"
output_label_dir = "/kaggle/working/pedri-dataset/labels/train"

# Create output directories
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Process each image
for img_file in os.listdir(input_img_dir):
    if not img_file.endswith('.jpg'):
        continue
    
    img_path = os.path.join(input_img_dir, img_file)
    label_path = os.path.join(input_label_dir, img_file.replace('.jpg', '.txt'))
    
    # Load image
    image = cv2.imread(img_path)
    if image is None:
        print(f"Failed to load image: {img_path}")
        continue
    
    # Load bounding boxes and classes
    bboxes = []
    classes = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                label, x, y, w, h = map(float, line.split())
                bboxes.append([x, y, w, h])
                classes.append(int(label))
    else:
        print(f"Label file not found: {label_path}")
        continue

    # Apply augmentation
    augmented = transform(image=image, bboxes=bboxes, class_labels=classes)
    
    # Save augmented image and labels
    aug_img_path = os.path.join(output_img_dir, f"aug_{img_file}")
    aug_label_path = os.path.join(output_label_dir, f"aug_{img_file.replace('.jpg', '.txt')}")
    
    cv2.imwrite(aug_img_path, augmented['image'])
    with open(aug_label_path, "w") as f:
        for cls, bbox in zip(augmented['class_labels'], augmented['bboxes']):
            f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")
    
    print(f"Augmented and saved: {aug_img_path}")

## 2. Setup YOLOv5

Clones YOLOv5 and installs dependencies.

In [2]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt

## 3. Define Training Parameters

Sets up variables for training.

In [3]:
PROJECT_NAME = "pedri_detection"
BASE_MODEL = "yolov5l.pt"
TRAIN_BATCH = 64
TRAIN_EPOCHS = 100
VAL_BATCH = 64

## 4. Create Data Configuration

Generates `data.yaml` for YOLOv5.

In [5]:
import yaml

# Define paths
train_path = "/kaggle/input/pedri-dataset/images/train"
val_path = "/kaggle/input/pedri-dataset/images/val"
test_path = "/kaggle/input/pedri-dataset/images/test"

# Write data.yaml
with open("data.yaml", "w") as file:
    yaml.dump({
        "train": train_path,
        "val": val_path,
        "test": val_path,  # Note: Matches your original code, adjust if test differs
        "nc": 2,
        "names": ["Pedri", "Others"]
    }, stream=file, default_flow_style=None)

## 5. Train the Model

Trains YOLOv5 with frozen layers.

In [6]:
!python yolov5/train.py --batch $TRAIN_BATCH --epochs $TRAIN_EPOCHS --data "data.yaml" --weights $BASE_MODEL --project $PROJECT_NAME --name 'feature_extraction' --cache --freeze 12

## 6. Detect Pedri in Video

Runs detection on `barca-video.mp4`.

In [7]:
!python yolov5/detect.py --weights /kaggle/working/pedri_detection/feature_extraction/weights/best.pt --img 640 --conf 0.25 --source /kaggle/input/pedri-dataset/barca-video.mp4 --project detect-runs --name pedri-detect --line-thickness 3 --save-txt --save-conf

## Notes
- **Paths**: Kaggle-specific (`/kaggle/input/`, `/kaggle/working/`). Adjust for local use.
- **Output**: Video saved to `detect-runs/pedri-detect/barca-video.mp4`.
- **Improvements**: Add more game footage, refine detection with DeepSORT (see README).